# Contents of Notebook: 

The contents of this notebook details the models applied to the enhancer gene link (ABC.Score) + neural network embeddings from the Basset 2016 Model <br> 


[Regression dataset](#linearregdataset)
<a href='#linearregdataset'></a>

I explored using Linear models like Linear Regression, Lasso, Ridge as well as Nonlinear models like Random Forest Regressor and Gradient Boosted Trees. 
[Model analysis overview](#linearmodeloverview)
<a href='#linearmodeloverview'></a>

### Models: 
1. [Linear Regression Base Model](#linearbase)<a href='#linearbase'></a> with default parameters 
2. [Lasso](#lasso)<a href='#lasso'></a> 
3. [Ridge Base Model](#ridge)<a href='#ridge'></a>
4. [Ridge w/ Parameter Tuning](#bestridge)<a href='#bestridge'></a>
5. [Random Forest Base Model](#rfrbase)<a href='#rfrbase'></a>
6. [Boosting algorithm with Random Forest model](#rfrboosting)<a href='#rfrboosting'></a>
Essentially, we applied a boosting algorithm where at each iteration the model fits the residual in the hopes that the final residual ends up being "noise" that the model can no longer learn from
7. [Random Forest Model after Classification](#rfrclassification)<a href='#rfrclassification'></a>
8. [Random Forest Model Base Model](#rfrbaseclass)<a href='#rfrbaseclass'></a>
9. [Random Forest Model w/o prior classification](#bestRFRnoclass)<a href='#bestRFRnoclass'></a>
10. [Gradient Boosting Regressor](#GBRbase)<a href='#GBRbase'></a> with default parameters
11. [Gradient Boosting Regressor w/ Parameter Tuning](#GBRbest)<a href='#GBRbest'></a>

After this initial test on the various models, I decided to look deeper into the data distribution. 

The neural network embeddings are from a sequence to chromatin accessibility Basset model from David Kelley's 2016 Paper. 

In the following notebook, I try to answer some fundamental questions about the dataset such as : 
1. Can we artificially curate the dataset to allow better fitting for linear models 
2. Why is there a characteristic vertical line present in the fitting of the following models 
--- the correlation graphs of the models used in this notebook had a vertical line that corresponded to the y-intercept of the linear model 


#### Import necessary libraries

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import pickle
import joblib
import sklearn
import scipy
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
%matplotlib inline

<a id='datapaths'></a>
#### Data paths

In [ ]:
labels = np.load("/mnt/lab_data2/kmualim/enhancer_gene_anlysis/datasets/all_dataset/gene_labels_all.npy", allow_pickle=True)
express_1 = pickle.load(open("/mnt/lab_data2/kmualim/enhancer_gene_anlysis/datasets/all_dataset/embeddings_all_w_gene.p","rb"))
expression = pd.read_csv("/mnt/lab_data2/kmualim/enhancer_gene_anlysis/datasets/K562.Genes.TPM.txt", sep=" ", header=None)
expression.head()

<a id='dataprocessing'></a>
#### Functions for combining and splitting data inputs 

In [ ]:
## Appending labels to expression table 
## 0 for non-expressed, 0 for expressed
label_1=[]
for i in expression[1]: 
    if i>1: 
        label_1.append(1)
    else: 
        label_1.append(0)

expression[2] = label_1

In [ ]:
from sklearn.model_selection import train_test_split  

def get_expression(express_dict, y_valid_labels): 
    x_val=[]
    for i in y_valid_labels[0]:
        x_val.append(express_dict[i])
    x_values = np.vstack(x_val)
    return x_values
    
def get_list(y_valid):
    y_valid_el = []
    for i in y_valid: 
        y_valid_el.append(i)
    return y_valid_el

# creating feature spaces 
def get_train_valid_split(labels, expression_dict, expression, classify=False):
    y_train, y_rem = train_test_split(labels, test_size=0.2, random_state=42, shuffle=True)
    y_valid, y_test = train_test_split(y_rem, test_size=0.25, random_state=42, shuffle=True)
    y_train_el = get_list(y_train) 
    y_valid_el = get_list(y_valid) 
    y_test_el = get_list(y_test) 
    y_train_labels = expression[expression[0].isin(y_train_el)]
    y_valid_labels = expression[expression[0].isin(y_valid_el)]
    y_test_labels = expression[expression[0].isin(y_test_el)]
    x_train = get_expression(expression_dict, y_train_labels)
    x_valid = get_expression(expression_dict, y_valid_labels)
    x_test = get_expression(expression_dict, y_test_labels)
    #if classify: 
    y_train_c, y_valid_c, y_test_c = get_class_labels(y_train_labels, y_valid_labels, y_test_labels)
    #else: 
    y_train_r, y_valid_r, y_test_r = get_reg_labels(y_train_labels, y_valid_labels, y_test_labels)
    return x_train, x_valid, x_test, y_train_r, y_valid_r, y_test_r, y_train_c, y_valid_c, y_test_c

def get_class_labels(y_train_labels, y_valid_labels, y_test_labels):
        y_train = y_train_labels[2].values
        y_valid = y_valid_labels[2].values
        y_test = y_test_labels[2].values 
        return y_train, y_valid, y_test
    
def get_reg_labels(y_train_labels, y_valid_labels, y_test_labels):
        y_train = y_train_labels[1].values
        y_valid = y_valid_labels[1].values
        y_test = y_test_labels[1].values
        return y_train, y_valid, y_test
    
    
    
%matplotlib inline
from scipy.stats import spearmanr,pearsonr
import matplotlib.pyplot as plt

def plot(y_valid_preds, y_valid): 
    print("Spearman R", spearmanr(y_valid_preds,np.arcsinh(y_valid)))
    print("Pearson R", pearsonr(y_valid_preds,np.arcsinh(y_valid)))
    plt.plot(y_valid_preds, np.arcsinh(y_valid), 'ro', markersize=2)
    plt.xlabel('Predictions')
    plt.ylabel('Actual')
    plt.title('Correlation graph')
    return plt

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = mean_squared_error(np.arcsinh(test_labels), predictions)
    plt = plot(predictions, test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    print('Model Performance')
    print('MSError: {:0.4f} '.format(errors))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return errors, plt #accuracy

<a id='datafiles'></a>
#### Data files and corresponding names

In [ ]:
x_train, x_valid, x_test, y_train, y_valid, y_test, y_train_labels, y_valid_labels, y_test_labels = get_train_valid_split(labels, express_1, expression, classify=True)

In [ ]:
#np.savez("train_classification_labels_abc_large.npz",train_preds)

<a id='linearmodeloverview'></a>
#### Overview of analysis [ABC.Score]
 
Linear Regression : <br>

SPEARMAN train,valid: 0.58197, 0.51572 <br>
PEARSON train, valid: 0.541520, 0.43851 <br>

Lasso: <br>

alpha=0.001 <br>
../scripts/ABC.Score_lascv.joblib <br>
SPEARMAN train, valid: 0.5404, 0.5288 <br>
PEARSON train, valid: 0.4479, 0.4259 <br>

Ridge : <br>

Alpha = 0.0001 <br>
../scripts/test_models/ABC.Score_ridgecv_0.0001-0.01.joblib <br>
SPEARMAN train, valid: 0.55974, 0.54069 <br>
PEARSON train, valid: 0.51249, 0.47275 <br>


Random Forest Regressor <br>
[BASE MODEL]: <br>
SPEARMAN train, valid, test:0.88130, 0.51643, 0.5678 <br>
PEARSON train, valid, test : 0.9280, 0.49752, 0.5409 <br>

[BEST MODEL AUGUST 5] : ../scripts/ABC.Score_all_RFR_cv10.joblib <br>
SPEARMAN train, valid = 0.73176, 0.5694 <br>
PEARSON train,valid = 0.738383, 0.5581036 <br>


GRADIENT BOOSTED TREES: <br>

GBT/GBR_base_ABC_large.joblib <br>
[BASE MODEL]: <br>
SPEARMAN train, valid, test: 0.63468, 0.571675 <br>
PEARSON train, valid, test : 0.639225, 0.581997 <br>

[BEST MODEL AUGUST 5] :../scripts/ABC.Score_all_GBT_cv10.joblib <br>
{'learning_rate': 0.05,
 'max_depth': 18,
 'max_features': 'sqrt',
 'min_samples_leaf': 15,
 'min_samples_split': 15,
 'n_estimators': 83}

SPEARMAN train, valid, test: 0.86290, 0.55637 <br>
PEARSON train, valid, test : 0.896418, 0.56607 <br>


In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

<a id='linearregdataset'></a>
#### path to Linear regression dataset

In [ ]:
x_train_reg, x_valid_reg, x_test_reg, y_train_reg, y_valid_reg, y_test_reg, y_train_labels, y_valid_labels, y_test_labels = get_train_valid_split(labels, express_1, expression, classify=False)


In [ ]:
# grabbing all training_data 
np.savez("datasets/train_valid_test_all/x_train_reg_all.npz", x_train_reg)
np.savez("datasets/train_valid_test_all/x_valid_reg_all.npz", x_valid_reg)
np.savez("datasets/train_valid_test_all/x_test_reg_all.npz", x_test_reg)
np.savez("datasets/train_valid_test_all/y_train_reg_all.npz", y_train_reg)
np.savez("datasets/train_valid_test_all/y_valid_reg_all.npz", y_valid_reg)
np.savez("datasets/train_valid_test_all/y_test_reg_all.npz", y_test_reg)
np.savez("datasets/train_valid_test_all/train_gene_labels_reg_all.npz", y_train_labels)
np.savez("datasets/train_valid_test_all/valid_gene_labels_reg_all.npz", y_valid_labels)
np.savez("datasets/train_valid_test_all/test_gene_labels_reg_all.npz", y_test_labels)

In [ ]:
def plot_after_class(y_valid_preds, y_valid): 
    print("Spearman R", spearmanr(y_valid_preds,np.arcsinh(y_valid)))
    print("Pearson R", pearsonr(y_valid_preds,np.arcsinh(y_valid)))
    plt.plot(y_valid_preds, np.arcsinh(y_valid), 'ro', markersize=2)
    plt.xlabel('Predictions')
    plt.ylabel('Actual')
    plt.title('Correlation graph')
    return plt

def get_number_of_zeros(test_labels, preds): 
    num_test_zero=0
    num_pred_zero=0
    for i in range(len(test_labels)): 
        if test_labels[i]==0 and preds[i]!=0:
            num_test_zero+=1
        if test_labels[i]!=0 and preds[i]==0:
            num_pred_zero+=1
    return num_test_zero, num_pred_zero
    
def eval_after_class(model, test_features, test_labels, test_class_labels, include_zeros=True):
    predictions = model.predict(test_features)
    errors = mean_squared_error(np.arcsinh(test_labels), (predictions*test_class_labels))
    final_preds = predictions*test_class_labels
    num_test_zero, num_pred_zero = get_number_of_zeros(test_labels, test_class_labels)
    if include_zeros:
        plt = plot_after_class(final_preds, test_labels)
    else: 
        plt = plot_after_class(final_preds[final_preds!=0], test_labels[final_preds!=0])
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    print('Model Performance')
    print('MSError: {:0.4f} degrees.'.format(errors))
    
    print("Number of Predictions!=0 and Actual==0:", num_test_zero)
    print("Number of Predictions==0 and Actual!=0:", num_pred_zero)
    print("Number of total Predictions:", len(final_preds))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return errors, predictions, plt #accuracy

Grabbing only the labels that are 1 "expressed" 
use as input for linear regression 

Reduces training data from ~16k to just ~7k 

<a id='expressedgenespath'></a>
#### paths to expressed genes datafile

In [ ]:
np.savez("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed.npz", train_big)
np.savez("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed_labels.npz", train_labels_big)

In [ ]:
train_big = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed.npz")['arr_0']
train_labels_big = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed_labels.npz")['arr_0']

In [ ]:
x_train_big, x_valid_big, y_train_big, y_valid_big = train_test_split(train_big, train_labels_big, test_size=0.2, random_state=42)

In [ ]:
print(len(train_big))

linreg_AFTERCLASS trained on ~6k examples

In [ ]:
joblib.dump(linreg_AFTERCLASS, open("linreg/linreg_abc_large_after_classification_TPM1.joblib", "wb"))

Getting y-intercept of linear regression

In [ ]:
linreg_AFTERCLASS = joblib.load("linreg/linreg_abc_large_after_classification_TPM1.joblib")
linreg_AFTERCLASS.intercept_

Plotting the distribution of arcsinh(TPM) values , manually picking out only the values where TPM != 0 

In [ ]:
# a lot of the zeros are also from the arcsinh of the thing 
#print(len(y_train_reg[y_train_reg==0]))
plt.hist(np.arcsinh(y_train_reg[y_train_reg!=0]), bins=100)
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution.pdf")

Y_TRAIN LABELS are np.arcsinh(TPM) <br>
Looking at the distribution, thresholding labels to just consider np.arcsinh(TPM) values above 0.5

In [ ]:
def plot_th(y_valid_preds, y_valid): 

    plt.plot(y_valid_preds, y_valid, 'ro', markersize=2)
    plt.xlabel('Predictions')
    plt.ylabel('Actual')
    plt.title('Correlation graph')
    return plt

def eval_th(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = mean_squared_error(test_labels,predictions)
    print("Spearman R", spearmanr(predictions,test_labels))
    print("Pearson R", pearsonr(predictions, test_labels))
    #plt = plot_th(predictions, test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    print('Model Performance')
    print('MSError: {:0.4f} '.format(errors))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return errors#, plt #accuracy

In [ ]:
y_val = np.arcsinh(y_train_reg)
rel = y_val[y_val>0.5]
x_train_rel = x_train_reg[y_val>0.5]

In [ ]:
y_val_1 = np.arcsinh(y_valid_reg)
y_val_rel = y_val_1[y_val_1>0.5]
x_valid_rel = x_valid_reg[y_val_1>0.5]

In [ ]:
print(len(rel))
print(len(y_val_rel))

In [ ]:
plt.hist(rel, bins=100)
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_0.5TPM.pdf")

Fitting linear regression using only TPM y values above a threshold of 0.5

In [ ]:
linreg_th = LinearRegression()
linreg_th.fit(x_train_rel, rel)

In [ ]:
linreg_th.intercept_

In [ ]:
_, plt = eval_th(linreg_th, x_train_rel, rel)

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_0.5_train_linreg.pdf")

In [ ]:
_, plt = evaluate(linreg_th, x_valid_rel, y_val_rel)
plt.xlim(0,10)

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_0.5_valid_linreg.png")

In [ ]:
joblib.dump(linreg_th, open("linreg/linreg_threshold_0.5_abc_large.joblib", "wb"))

Y_TRAIN LABELS are np.arcsinh(TPM) <br>
Thresholding labels to just consider np.arcsinh(TPM) values above 1.0

In [ ]:
y_val = np.arcsinh(y_train_reg)
rel = y_val[y_val>1.0]
x_train_rel = x_train_reg[y_val>1.0]

y_val_1 = np.arcsinh(y_valid_reg)
y_val_rel = y_val_1[y_val_1>1.0]
x_valid_rel = x_valid_reg[y_val_1>1.0]

In [ ]:
plt.hist(rel, bins=100)
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_1.0.png")

Fitting linear regression using only TPM y values above a threshold of 1.0 

In [ ]:
linreg_th_1 = LinearRegression()
linreg_th_1.fit(x_train_rel, rel)

In [ ]:
linreg_th_1.intercept_

In [ ]:
_, plt = eval_th(linreg_th_1, x_train_rel, rel)

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_1.0_train_linreg.png")

In [ ]:
_, plt = evaluate(linreg_th_1, x_valid_rel, y_val_rel)
plt.xlim(0,6)

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_1.0_valid_linreg.png")

In [ ]:
joblib.dump(linreg_th_1, open("linreg/linreg_threshold_1_abc_large.joblib", "wb"))

CORRELATION GRAPH ON TRAIN

<a id='linearbase'></a>
### Linear Regression BASE MODEL (ABC.Score + Neural network embeddings)

In [ ]:
linreg = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/test_models/ABC.Score_linreg.joblib")

In [ ]:
print(linreg.intercept_)

In [ ]:
_, plt  = evaluate(linreg, x_train, np.arcsinh(y_train))

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_1.0_train_linreg.png")

CORRELATION GRAPH ON VALID

In [ ]:
_, plt = evaluate(linreg, x_valid, np.arcsinh(y_valid))

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_threshold_1.0_valid_linreg.png")

<a id='lasso'></a>
## Lasso

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
lasso = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/test_models/ABC.Score_lascv.joblib")

In [ ]:
lasso.alpha_

CORRELATION GRAPH FOR TRAIN

In [ ]:
_, plt = evaluate(lasso, x_train, np.arcsinh(y_train))

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_train_lasso.png")

CORRELATION GRAPH FOR VALID 

In [ ]:
_, plt = evaluate(lasso, x_valid, np.arcsinh(y_valid))

In [ ]:
plt.savefig("/oak/stanford/groups/akundaje/projects/egl_analysis/images/abc_large_y_distribution_valid_lasso.png")

<a id='ridge'></a>
## Ridge

#### BASE 

In [ ]:
ridge = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/test_models/ABC.Score_ridge_base.joblib")

CORRELATION GRAPH ON TRAIN

In [ ]:
evaluate(ridge, x_train, np.arcsinh(y_train))

CORRELATION GRAPH ON VALID

In [ ]:
evaluate(ridge, x_valid, np.arcsinh(y_valid))

<a id='bestridge'></a>
#### BEST RIDGE CV MODEL  

In [ ]:
model_ridge = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/test_models/ABC.Score_ridgecv_0.0001-0.01.joblib")

In [ ]:
model_ridge.alpha_

CORRELATION GRAPH ON TRAIN

In [ ]:
evaluate(model_ridge, x_train, np.arcsinh(y_train))

CORRELATION GRAPH ON VALID

In [ ]:
evaluate(model_ridge, x_valid, np.arcsinh(y_valid))

<a id='rfrbase'></a>
## Random Forest Regressor

#### BASE MODEL

In [ ]:
regressor

In [ ]:
regressor = RandomForestRegressor()
print("Modeling is fitting ...")
regressor.fit(x_train,np.arcsinh(y_train))
y_valid_preds = regressor.predict(x_valid)

CORRELATION GRAPH ON TRAIN

In [ ]:
y_train_preds = regressor.predict(x_train)
plot(y_train_preds, y_train)

CORRELATION GRAPH ON VALID

In [ ]:
plot(y_valid_preds, y_valid)

CORRELATION GRAPH ON TEST

In [ ]:
y_test_preds = regressor.predict(x_test)
plot(y_test_preds, y_test)

In [ ]:
joblib.dump(regressor, open("RFR/RFR_base_ABC_large.joblib", "wb"))

<a id='rfrboosting'></a>
### Random Forest Regressor Residual Iterations

At 20 iterations

In [ ]:
residual_RFR = np.load("/mnt/lab_data2/kmualim/Jamboree_data/scripts/src/RF_best_iteration_residual.npz")['arr_0']

In [ ]:
actual_RFR = np.load("/mnt/lab_data2/kmualim/Jamboree_data/scripts/src/RF_best_iteration.npz")['arr_0']

In [ ]:
plt.plot(actual_RFR, np.arcsinh(train_big_labels),  'ro')
plt.xlabel("Predictions")
plt.ylabel("Actual")

In [ ]:
plt.plot(residual_RFR, np.arcsinh(train_big_labels),  'ro')
plt.xlabel("Predictions")
plt.ylabel("Actual")

In [ ]:
residual_RFR.shape

<a id='rfrclassification'></a>
### Evaluating Random Forest Regressor after prior classification
-- final regressor is on expressed genes
-- after logistic regression 

In [ ]:
Aug11_class_model = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/Aug11_ABC.Score_after_classification_all_RFR_cv10.joblib")

In [ ]:
train_labels = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/train_classification_labels_abc_large.npz")['arr_0']

In [ ]:
test_labels = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/test_classification_labels_abc_large.npz")['arr_0']

In [ ]:
aug11_model = RandomForestRegressor(bootstrap= True,
max_depth= 51,
max_features= 'sqrt',
min_samples_leaf= 15,
min_samples_split= 35,
n_estimators= 178)

In [ ]:
Aug11_class_model.best_params_

In [ ]:
aug11_model.fit(x_train_reg[train_labels!=0], y_train_reg[train_labels!=0])

run_time =~ 78 minutes

In [ ]:
eval_after_class(Aug11_class_model, x_train_reg, y_train_reg, train_labels)

In [ ]:
## Not considering the zeros 
eval_after_class(Aug11_class_model, x_train_reg, y_train_reg, train_labels, include_zeros=False)

Plotting correlation graph for test samples

In [ ]:
eval_after_class(Aug11_class_model, x_test_reg, y_test_reg, test_labels)

In [ ]:
eval_after_class(Aug11_class_model, x_test_reg, y_test_reg, test_labels, include_zeros=False)

<a id='rfrbaseclass'></a>
#### Looking at how the BASE random forest model performs after prior classification

In [ ]:
train_big = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed.npz")['arr_0']
train_big_labels = np.load("/mnt/lab_data2/kmualim/Jamboree_data/notebooks/datasets/7580_abc_large_expressed_labels.npz")['arr_0']

In [ ]:
base_model = RandomForestRegressor()
base_model.fit(train_big, np.arcsinh(train_big_labels))

In [ ]:
joblib.dump(base_model ,open("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/test_models/ABC.Score_rfr_base_model.joblib", "wb"))

In [ ]:
base_model = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/models/abc_all/test_models/ABC.Score_rfr_base_model.joblib")

In [ ]:
eval_after_class(base_model, x_test_reg, y_test_reg, test_labels)

In [ ]:
eval_after_class(base_model, x_test_reg, y_test_reg, test_labels, include_zeros=False)

<a id='bestRFRnoclass'></a>
### Best RFR model without classification step

In [ ]:
Aug9_model = joblib.load("/mnt/lab_data2/kmualim/Jamboree_data/scripts/test_models/Aug9_ABC.Score_all_RFR_cv10.joblib")

In [ ]:
Aug9_model.best_params_

In [ ]:
evaluate(Aug9_model, x_train_reg, np.arcsinh(y_train_reg))

In [ ]:
evaluate(Aug9_model, x_valid_reg, np.arcsinh(y_valid_reg))

<a id='GBRbase'></a>
### Gradient Boosted Regressor

#### BASE MODEL

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor 

In [ ]:
GBR = GradientBoostingRegressor()
GBR.fit(x_train, np.arcsinh(y_train))

In [ ]:
evaluate(GBR, x_train, np.arcsinh(y_train))

In [ ]:
evaluate(GBR, x_valid, np.arcsinh(y_valid))

In [ ]:
joblib.dump(GBR, open("GBT/GBR_base_ABC_large.joblib", "wb"))

<a id='GBRbest'></a>
#### Model Aug5 <br> 


In [ ]:
GBR_aug5 = joblib.load("../scripts/ABC.Score_all_GBT_cv10.joblib")

In [ ]:
GBR_aug5.best_params_

In [ ]:
evaluate(GBR_aug5, x_train, np.arcsinh(y_train))

In [ ]:
evaluate(GBR_aug5, x_valid, np.arcsinh(y_valid))

#### Understanding the feature importances of the models

In [ ]:
x = np.arange(0,1000,1)

In [ ]:
lasso.coef_

In [ ]:
import plotly.graph_objects as go
fig_lasso = go.Figure(data=go.Scatter(x=x, y=lasso.coef_, mode='markers'))
fig_lasso.show()

In [ ]:
fig_ridge = go.Figure(data=go.Scatter(x=x, y=model_ridge.coef_, mode='markers'))
fig_ridge.show()

In [ ]:
fig_rfr = go.Figure(data=go.Scatter(x=x, y=aug11_model.feature_importances_, mode='markers'))
fig_rfr.show()